# Text Generation Experiments

In [1]:
import torch
import train
import json
from dataset import TextDataset
from model import TextGenerationModel
import numpy as np
import random
import ast
import os

class Namespace:
    def __init__(self, **kwargs):
        self.__dict__.update(kwargs)
    def __str__(self):
        return json.dumps(self.__dict__, indent=4)

In [2]:
def load_model(path):
    with open (os.path.join(path, 'hyperparams.txt'), 'r') as file:
        hp = eval(file.read())
    hp = hp.__dict__
    checkpoint = torch.load(os.path.join(path, 'checkpoint.pth.tar'), map_location='cpu')

    dataset = TextDataset(hp['txt_file'], hp['seq_length'], hp['batch_size'], hp['train_steps'])

    model = TextGenerationModel(vocabulary_size=dataset.vocab_size,
                                lstm_num_hidden=hp['lstm_num_hidden'],
                                lstm_num_layers=hp['lstm_num_layers'],
                                dropout=1-hp['dropout_keep_prob'],
                                temperature=hp['temperature']
                               )
    model.load_state_dict(checkpoint['state_dict'])

    return model, dataset, hp

def sample_sequence_30(model, dataset, hp, seed_char=None):
    if seed_char:
        seed_char = dataset._char_to_ix[seed_char]
    else:
        seed_char = np.random.choice(range(dataset.vocab_size))
    with torch.no_grad():
        model.eval()

        # convert char to one-hot encoding
        ins = torch.zeros((1, 1, dataset.vocab_size)).to(dtype=torch.float)
        ins[0, 0, seed_char] = 1.0

        # initialize hidden state and cell
        h_t = torch.zeros(hp['lstm_num_layers'], 1, hp['lstm_num_hidden'], dtype=torch.float)
        c_t = torch.zeros(hp['lstm_num_layers'], 1, hp['lstm_num_hidden'], dtype=torch.float)

        # predict the next character and feed back into model
        predictions = [seed_char]
        for t in range(hp['seq_length']):
            # compute log-odds
            outs, (h_t, c_t), _ = model(ins, (h_t, c_t))
            # greedily select character according to prediction
            pred_char = torch.argmax(outs, dim=1)
            # append to generated sequence
            predictions.append(pred_char.item())
            # feed back into model
            ins = torch.zeros((1, 1, dataset.vocab_size)).to(dtype=torch.float)
            ins[0, 0, pred_char.item()] = 1.0
    print(dataset.convert_to_string(predictions))
    
def sample_sequence_longer(model, dataset, hp, seed_sentence, sequence_length):
    with torch.no_grad():
        model.eval()

        # convert sentence to one-hot representation
        tokenized_sentence = [dataset._char_to_ix[char] for char in seed_sentence]
        one_hot_dim = (len(tokenized_sentence), 1, dataset.vocab_size)
        one_hot_sentence = torch.zeros(*one_hot_dim, dtype=torch.float).scatter_(2, torch.tensor(tokenized_sentence).reshape(-1,1,1), 1.0)
        
        # extract the last character as the initial input to character-by-character sampling
        ins = one_hot_sentence[-1, :,:].view(1,1,dataset.vocab_size)
        # exclude the last character from the sentence on the first forward pass
        # to get the hidden state that should be passed with the last character.
        one_hot_sentence = one_hot_sentence[:-1, :,:]
        

        # obtain hidden state over seed sentence without last character.
        _, (h_t, c_t), _ = model(one_hot_sentence)
        
        # predict the next character and feed back into model
        predictions = tokenized_sentence
        for t in range(sequence_length):
            # compute log-odds
            outs, (h_t, c_t), _ = model(ins, (h_t, c_t))
            # greedily select character according to prediction
            pred_char = torch.argmax(outs, dim=1)
            # append to generated sequence
            predictions.append(pred_char.item())
            # feed back into model
            ins = torch.zeros((1, 1, dataset.vocab_size)).to(dtype=torch.float)
            ins[0, 0, pred_char.item()] = 1.0
    print(dataset.convert_to_string(predictions))

## US Constitution Text Generation with T=0.5

In [14]:
model, dataset, hp = load_model('models/2018-09-29 01-18.us_constitution')

Initialize dataset with 2882 characters, 57 unique.


In [15]:
sample_sequence_30(model, dataset, hp, 'C')

Constitution,
nor prohibited by


In [22]:
sample_sequence_longer(model, dataset, hp, 'No person shall be', 242)


No person shall be held to answer for a capital, or otherwise infamous crime,
unless on a presentment or indictment of a Grand Jury, except in cases arising
in the land or naval forces, or in the Militia, when in actual service
in time of War or public danger;


## US Constitution Text Generation with T=1.0

In [42]:
model, dataset, hp = load_model('models/2018-09-30 20-03.us_constitution')

Initialize dataset with 2882 characters, 57 unique.


In [43]:
sample_sequence_30(model, dataset, hp, 'C')

Constitution of the United Stat


In [44]:
sample_sequence_longer(model, dataset, hp, 'No person shall be', 242)

No person shall be held to answer for a capital, or otherwise infamous crime,
unless on a presentment or indictment of a Grand Jury, except in cases arising
in the land or naval forces, or in the Militia, when in actual service
in time of War or public danger;


## US Constitution Text Generation with T=1.5

In [53]:
model, dataset, hp = load_model('models/2018-09-29 02-02.us_constitution')

Initialize dataset with 2882 characters, 57 unique.


In [54]:
sample_sequence_30(model, dataset, hp, 'C')

Constitution, of certain rights


In [55]:
sample_sequence_longer(model, dataset, hp, 'No person shall be', 242)

No person shall be held to answer for a capital, or otherwise infamous crime,
unless on a presentment or indictment of a Grand Jury, except in cases arising
in the land or naval forces, or in the Militia, when in actual service
in time of War or public danger;


## US Constitution Text Generation with T=2.0

In [49]:
model, dataset, hp = load_model('models/2018-09-29 01-10.us_constitution')

Initialize dataset with 2882 characters, 57 unique.


In [50]:
sample_sequence_30(model, dataset, hp, 'C')


Congress September 25, 1789
Rat


In [51]:
sample_sequence_longer(model, dataset, hp, 'No person shall be', 216)

No person shall be held to answer for a redress of grievances.


II

A well-regulated militia, being necessary to the security of a free State,
the right of the people.

X

The powers not delegated to the United States Bill of Rights.


## Random Sample with exogenous T=0.5,1.0,1.5,2.0

In [13]:
# Currently not possible because the network is outputting only log-odds, no energies

# seed_char = np.random.choice(range(dataset.vocab_size))
# with torch.no_grad():
#     model.eval()
#     for T in [0.5,1.0,1.5,2.0]:
#         # convert char to one-hot encoding
#         ins = torch.zeros((1, 1, dataset.vocab_size)).to(dtype=torch.float)
#         ins[0, 0, seed_char] = 1.0

#         # initialize hidden state and cell
#         h_t = torch.zeros(hp['lstm_num_layers'], 1, hp['lstm_num_hidden']).to(dtype=torch.float)
#         c_t = torch.zeros(hp['lstm_num_layers'], 1, hp['lstm_num_hidden']).to(dtype=torch.float)

#         # predict the next character and feed back into model
#         predictions = []
#         for t in range(hp['seq_length']):
#             # compute log-odds
#             outs, (h_t, c_t) = model(ins, (h_t, c_t))
#             # compute annealed distribution
#             outs = torch.softmax(outs / T, dim=2)
#             # sample character according to prediction
#             pred_char = np.random.choice(np.array(range(dataset.vocab_size)), p=outs.numpy().squeeze())
#             # append to generated sequence
#             predictions.append(pred_char.item())
#             # feed back into model
#             ins = torch.zeros((1, 1, dataset.vocab_size)).to(dtype=torch.float)
#             ins[0, 0, pred_char.item()] = 1.0
#         print(T + ':' + ' ' + dataset.convert_to_string(predictions))